In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install tensorflow

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

In [4]:
train_dir = '/content/drive/MyDrive/Dataset/train'
val_dir = '/content/drive/MyDrive/Dataset/val'
test_dir = '/content/drive/MyDrive/Dataset/test'

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_val_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_val_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 5226 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(
    train_generator,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size), # Ensure steps_per_epoch is at least 1
    epochs=10,
    validation_data=validation_generator,
    validation_steps=max(1, validation_generator.samples // validation_generator.batch_size) # Ensure validation_steps is at least 1
)

Epoch 1/10
163/163 [==============================] - 99s 608ms/step - loss: 0.2280 - accuracy: 0.9082 - val_loss: 0.4099 - val_accuracy: 0.8750
Epoch 2/10
163/163 [==============================] - 96s 588ms/step - loss: 0.2047 - accuracy: 0.9178 - val_loss: 0.5084 - val_accuracy: 0.7500
Epoch 3/10
163/163 [==============================] - 93s 571ms/step - loss: 0.1893 - accuracy: 0.9234 - val_loss: 0.6351 - val_accuracy: 0.7500
Epoch 4/10
163/163 [==============================] - 93s 572ms/step - loss: 0.1756 - accuracy: 0.9313 - val_loss: 0.5280 - val_accuracy: 0.7500
Epoch 5/10
163/163 [==============================] - 94s 577ms/step - loss: 0.1633 - accuracy: 0.9369 - val_loss: 0.4905 - val_accuracy: 0.7500
Epoch 6/10
163/163 [==============================] - 92s 567ms/step - loss: 0.1647 - accuracy: 0.9376 - val_loss: 0.8288 - val_accuracy: 0.6875
Epoch 7/10
163/163 [==============================] - 95s 580ms/step - loss: 0.1465 - accuracy: 0.9434 - val_loss: 0.5018 - val_ac

In [17]:
# Continue training for an additional 15 epochs
history = model.fit(
    train_generator,
    steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),
    epochs=25,  # Total number of epochs, including the initial 10 and additional 15
    initial_epoch=10,  # Continue training from epoch 10
    validation_data=validation_generator,
    validation_steps=max(1, validation_generator.samples // validation_generator.batch_size)
)

Epoch 11/25
163/163 [==============================] - 96s 588ms/step - loss: 0.1377 - accuracy: 0.9494 - val_loss: 0.4736 - val_accuracy: 0.8125
Epoch 12/25
163/163 [==============================] - 91s 557ms/step - loss: 0.1196 - accuracy: 0.9526 - val_loss: 0.1885 - val_accuracy: 0.9375
Epoch 13/25
163/163 [==============================] - 94s 573ms/step - loss: 0.1204 - accuracy: 0.9532 - val_loss: 0.5815 - val_accuracy: 0.7500
Epoch 14/25
163/163 [==============================] - 92s 562ms/step - loss: 0.1299 - accuracy: 0.9523 - val_loss: 0.5784 - val_accuracy: 0.7500
Epoch 15/25
163/163 [==============================] - 92s 564ms/step - loss: 0.1146 - accuracy: 0.9551 - val_loss: 0.3944 - val_accuracy: 0.7500
Epoch 16/25
163/163 [==============================] - 93s 568ms/step - loss: 0.1042 - accuracy: 0.9588 - val_loss: 0.5272 - val_accuracy: 0.8125
Epoch 17/25
163/163 [==============================] - 92s 567ms/step - loss: 0.1072 - accuracy: 0.9571 - val_loss: 0.9407 -

In [18]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {test_acc}")

19/19 [==============================] - 134s 7s/step - loss: 0.5001 - accuracy: 0.8799
Test accuracy: 0.8799341917037964


In [19]:
# Save the entire model
model.save('pneumonia_detection_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
